# Execution num1: 

In [1]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.dqn.agent import Agent, Config
from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

/home/samuel/anaconda3/envs/myEnv3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/samuel/anaconda3/envs/myEnv3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/samuel/anaconda3/envs/myEnv3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/samuel/anaconda3/envs/myEnv3_7/lib

In [2]:
#%%
scenario = "basic"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "basic",
    "nWifi": 15,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:50877
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (2.849s)


Ns3Env parameters:
--nWifi: 15
--simulationTime: 60
--openGymPort: 50877
--envStepTime: 0.01
--seed: 42
--agentType: discrete
--scenario: basic
--dryRun: 0
Simulation started


Observation space shape: (1, 300)
Action space shape: (1, 1)


Simulation process id: 468168 (parent (waf shell) id: 468153)
Waiting for Python process to connect on port: tcp://localhost:50877
Please start proper Python Gym Agent


In [3]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr = 4e-4
config = Config(buffer_size=3*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr=lr, update_every=1)
agent = Agent(QNetworkTf, history_length, action_size=7, config=config)
agent.set_epsilon(0.9, 0.001, EPISODE_COUNT-2)

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        "Final",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"LR: {lr}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# # agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
'''logger = teacher.eval(agent,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        tags=tags,
                        parameters=hyperparams)
'''
# agent.save()


# %%

2023-02-13 02:28:57.092814: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2023-02-13 02:28:57.121765: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3491795000 Hz
2023-02-13 02:28:57.122553: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x557e3e57d690 executing computations on platform Host. Devices:
2023-02-13 02:28:57.122767: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dense instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





2023-02-13 02:28:59.834297: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.



Action space: 7
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


...

Ns3Env parameters:
--nWifi: 15
--simulationTime: 60
--openGymPort: 50153
--envStepTime: 0.01
--seed: 42
--agentType: discrete
--scenario: basic
--dryRun: 0
Simulation started
Simulation process id: 909440 (parent (waf shell) id: 909393)
Waiting for Python process to connect on port: tcp://localhost:50153
Please start proper Python Gym Agent
COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.com/sheila-janota/rl-in-wifi-with-queue-level/96dfe4b8d1724dc097d7f2a7f21e6150
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Chosen CW [89985]           : (16.0, 1024.0)
COMET INFO:     Cumulative reward [15]      : (2403.75146484375, 3174.3232421875)
COMET INFO:     Current throughput [89985]  : (19.300783157348633, 69.25048828125)
COMET INFO:     Fairness index [89985]      : (nan, nan)
COMET INFO:     Megabytes sent [89985]      : (0.2122189998626709, 2267.6630370020866)
COMET INFO:     Observati

Sent 2267.66 Mb/s.	Mean speed: 37.79 Mb/s	Episode 15/15 finished



COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
COMET INFO: Waiting for completion of the file uploads (may take several seconds)
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Still uploading
COMET INFO: Sti

Training finished.


'logger = teacher.eval(agent,\n                        simTime=simTime,\n                        stepTime=stepTime,\n                        history_length=history_length,\n                        tags=tags,\n                        parameters=hyperparams)\n'

# Execution num2: 

In [1]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.dqn.agent import Agent, Config
from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

/home/samuel/anaconda3/envs/myEnv3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/samuel/anaconda3/envs/myEnv3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/samuel/anaconda3/envs/myEnv3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/samuel/anaconda3/envs/myEnv3_7/lib

In [2]:
#%%
scenario = "basic"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "basic",
    "nWifi": 15,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:49059
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (4.119s)


Ns3Env parameters:
--nWifi: 15
--simulationTime: 60
--openGymPort: 49059
--envStepTime: 0.01
--seed: 42
--agentType: discrete
--scenario: basic
--dryRun: 0
Simulation started


Observation space shape: (1, 300)
Action space shape: (1, 1)


Simulation process id: 988225 (parent (waf shell) id: 988181)
Waiting for Python process to connect on port: tcp://localhost:49059
Please start proper Python Gym Agent


In [ ]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr = 4e-4
config = Config(buffer_size=3*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr=lr, update_every=1)
agent = Agent(QNetworkTf, history_length, action_size=7, config=config)
agent.set_epsilon(0.9, 0.001, EPISODE_COUNT-2)

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        "Final",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"LR: {lr}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# # agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
#logger = teacher.eval(agent,
#                        simTime=simTime,
#                        stepTime=stepTime,
#                        history_length=history_length,
#                        tags=tags,
#                        parameters=hyperparams)
# agent.save()


# %%

In [ ]:
COMET INFO: Comet.ml Experiment Summary: 
COMET INFO:   Data: 
COMET INFO:     url: https://www.comet.com/sheila-janota/rl-in-wifi-with-queue-level/75cd994f544348ceaeeffa060f0e9342 
COMET INFO:   Metrics [count] (min, max): 
COMET INFO:     Chosen CW [89985]           : (16.0, 1024.0) 
COMET INFO:     Cumulative reward [15]      : (2456.813720703125, 3166.90673828125) 
COMET INFO:     Current throughput [89985]  : (20.451234817504883, 65.49755096435547) 
COMET INFO:     Fairness index [89985]      : (nan, nan) 
COMET INFO:     Megabytes sent [89985]      : (0.245728000998497, 2263.1059050410986) 
COMET INFO:     Observation 0 [89985]       : (0.9332020816206932, 0.9341666666666667) 
COMET INFO:     Observation 1 [89985]       : (0.018178883788577436, 0.020230517468018055) 
COMET INFO:     Per-ep reward [89985]       : (126.97412109375, 3166.90673828125) 
COMET INFO:     Round megabytes sent [89985]: (0.01116899959743023, 11.839599609375) 
COMET INFO:     Round reward [89985]        : (0.014892578125, 0.640380859375) 
COMET INFO:     Speed [15]                  : (32.05888123571252, 37.71843175068498) 
COMET INFO:     Station count [89985]       : (15.0, 15.0) 
COMET INFO:     loss [89985]                : (0.0, 0.22322774678468704) 
COMET INFO:   Uploads: 
COMET INFO:     histogram3d: 89985 
COMET INFO: ---------------------------- 
 

Sent 2262.12 Mb/s.	Mean speed: 37.70 Mb/s	Episode 15/15 finished

# Execution num3:

In [1]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.dqn.agent import Agent, Config
from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

/home/samuel/anaconda3/envs/myEnv3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/samuel/anaconda3/envs/myEnv3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/samuel/anaconda3/envs/myEnv3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/samuel/anaconda3/envs/myEnv3_7/lib

In [2]:
#%%
scenario = "basic"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "basic",
    "nWifi": 15,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:46072
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (4.192s)


Ns3Env parameters:
--nWifi: 15
--simulationTime: 60
--openGymPort: 46072
--envStepTime: 0.01
--seed: 42
--agentType: discrete
--scenario: basic
--dryRun: 0
Simulation started


Observation space shape: (1, 300)
Action space shape: (1, 1)


Simulation process id: 1408193 (parent (waf shell) id: 1408149)
Waiting for Python process to connect on port: tcp://localhost:46072
Please start proper Python Gym Agent


In [3]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr = 4e-4
config = Config(buffer_size=3*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr=lr, update_every=1)
agent = Agent(QNetworkTf, history_length, action_size=7, config=config)
agent.set_epsilon(0.9, 0.001, EPISODE_COUNT-2)

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        "Final",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"LR: {lr}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# # agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
#logger = teacher.eval(agent,
#                        simTime=simTime,
#                        stepTime=stepTime,
#                        history_length=history_length,
#                        tags=tags,
#                        parameters=hyperparams)
# agent.save()


# %%

2023-02-15 10:32:45.921034: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2023-02-15 10:32:45.971962: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3491795000 Hz
2023-02-15 10:32:45.972628: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55b592179cc0 executing computations on platform Host. Devices:
2023-02-15 10:32:45.972648: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dense instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





2023-02-15 10:32:48.416837: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.



Action space: 7
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


...

Waiting for simulation script to connect on port: tcp://localhost:45054
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (3.202s)


Ns3Env parameters:
--nWifi: 15
--simulationTime: 60
--openGymPort: 45054
--envStepTime: 0.01
--seed: 42
--agentType: discrete
--scenario: basic
--dryRun: 0
Simulation started
Simulation process id: 2135656 (parent (waf shell) id: 2135630)
Waiting for Python process to connect on port: tcp://localhost:45054
Please start proper Python Gym Agent
COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.com/sheila-janota/rl-in-wifi-with-queue-level/4a17686428394544a1cf7c4a6588cf1f
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Chosen CW [89985]           : (16.0, 1024.0)
COMET INFO:     Cumulative reward [15]      : (2419.552490234375, 3168.947021484375)
COMET INFO:     Current throughput [89985]  : (17.8264217376709, 71.40618133544922)
COMET INFO:     Fairness index [89985]      : (nan, nan)
COMET INFO:     Megabytes sent [89985]      : (0.2680659890174866, 2264.334484770894)
COMET INFO:     Obse

Sent 2264.33 Mb/s.	Mean speed: 37.74 Mb/s	Episode 15/15 finished



COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


Training finished.


# Execution num4:

In [1]:
#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.dqn.agent import Agent, Config
from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

/home/samuel/anaconda3/envs/myEnv3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/samuel/anaconda3/envs/myEnv3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/samuel/anaconda3/envs/myEnv3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/samuel/anaconda3/envs/myEnv3_7/lib

In [2]:
#%%
scenario = "basic"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "basic",
    "nWifi": 15,
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:26442
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (2.851s)


Ns3Env parameters:
--nWifi: 15
--simulationTime: 60
--openGymPort: 26442
--envStepTime: 0.01
--seed: 42
--agentType: discrete
--scenario: basic
--dryRun: 0
Simulation started


Observation space shape: (1, 300)
Action space shape: (1, 1)


Simulation process id: 1780891 (parent (waf shell) id: 1780869)
Waiting for Python process to connect on port: tcp://localhost:26442
Please start proper Python Gym Agent


In [3]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr = 4e-4
config = Config(buffer_size=3*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr=lr, update_every=1)
agent = Agent(QNetworkTf, history_length, action_size=7, config=config)
agent.set_epsilon(0.9, 0.001, EPISODE_COUNT-2)

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        "Final",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"LR: {lr}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# # agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
#logger = teacher.eval(agent,
#                        simTime=simTime,
#                        stepTime=stepTime,
#                        history_length=history_length,
#                        tags=tags,
#                        parameters=hyperparams)
# agent.save()


# %%

2023-02-14 14:39:50.382489: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2023-02-14 14:39:50.403949: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3491795000 Hz
2023-02-14 14:39:50.404599: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x56008cb52690 executing computations on platform Host. Devices:
2023-02-14 14:39:50.404617: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dense instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





2023-02-14 14:39:52.796873: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.



Action space: 7
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


...

Waiting for simulation script to connect on port: tcp://localhost:39989
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (2.990s)


Ns3Env parameters:
--nWifi: 15
--simulationTime: 60
--openGymPort: 39989
--envStepTime: 0.01
--seed: 42
--agentType: discrete
--scenario: basic
--dryRun: 0
Simulation started
Simulation process id: 1715080 (parent (waf shell) id: 1715061)
Waiting for Python process to connect on port: tcp://localhost:39989
Please start proper Python Gym Agent
COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.com/sheila-janota/rl-in-wifi-with-queue-level/5c2c62d380784043911a6fb3fba8c8e3
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Chosen CW [89985]           : (16.0, 1024.0)
COMET INFO:     Cumulative reward [15]      : (2376.304443359375, 3165.71533203125)
COMET INFO:     Current throughput [89985]  : (18.79815673828125, 58.840572357177734)
COMET INFO:     Fairness index [89985]      : (nan, nan)
COMET INFO:     Megabytes sent [89985]      : (0.18987999856472015, 2261.7767177820206)
COMET INFO:     O

Sent 2258.01 Mb/s.	Mean speed: 37.63 Mb/s	Episode 15/15 finished



COMET INFO: Uploading stats to Comet before program termination (may take several seconds)


Training finished.


In [3]:
#%%
teacher = Teacher(env, 1, Preprocessor(False))

lr = 4e-4
config = Config(buffer_size=3*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr=lr, update_every=1)
agent = Agent(QNetworkTf, history_length, action_size=7, config=config)
agent.set_epsilon(0.9, 0.001, EPISODE_COUNT-2)

# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        "Final",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"LR: {lr}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# # agent.save()
logger = teacher.train(agent, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
#logger = teacher.eval(agent,
#                        simTime=simTime,
#                        stepTime=stepTime,
#                        history_length=history_length,
#                        tags=tags,
#                        parameters=hyperparams)
# agent.save()


# %%

2023-02-13 21:47:17.015568: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2023-02-13 21:47:17.067971: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3491795000 Hz
2023-02-13 21:47:17.084057: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55ffa2e05200 executing computations on platform Host. Devices:
2023-02-13 21:47:17.084477: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dense instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





2023-02-13 21:47:19.709627: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA_FLAGS) or set the envvar XLA_FLAGS=--xla_hlo_profile.



Action space: 7
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


...

Sent 2239.93 Mb/s.	Mean speed: 37.33 Mb/s	Episode 14/15 finished

14
Turning off noise


100%|██████████| 6300/6300 [1:01:59<00:00,  1.69it/s, curr_speed=38.22 Mbps, mb_sent=2258.01 Mb]


Waiting for simulation script to connect on port: tcp://localhost:37063
Please start proper ns-3 simulation script using ./waf --run "..."
Waf: Entering directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Waf: Leaving directory `/home/samuel/ns-allinone-3.29/ns-3.29/build'
Build commands will be stored in build/compile_commands.json
'build' finished successfully (3.784s)


Ns3Env parameters:
--nWifi: 15
--simulationTime: 60
--openGymPort: 37063
--envStepTime: 0.01
--seed: 42
--agentType: discrete
--scenario: basic
--dryRun: 0
Simulation started
Simulation process id: 1357698 (parent (waf shell) id: 1357658)
Waiting for Python process to connect on port: tcp://localhost:37063
Please start proper Python Gym Agent
COMET INFO: ----------------------------
COMET INFO: Comet.ml Experiment Summary:
COMET INFO:   Data:
COMET INFO:     url: https://www.comet.com/sheila-janota/rl-in-wifi-with-queue-level/81f4cc378146449e928216874f49f814
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     Chosen CW [89985]           : (16.0, 1024.0)
COMET INFO:     Cumulative reward [15]      : (2510.14404296875, 3163.138916015625)
COMET INFO:     Current throughput [89985]  : (19.41248321533203, 68.53564453125)
COMET INFO:     Fairness index [89985]      : (nan, nan)
COMET INFO:     Megabytes sent [89985]      : (0.20104999840259552, 2259.598648548126)
COMET INFO:     Observ

Sent 2258.01 Mb/s.	Mean speed: 37.63 Mb/s	Episode 15/15 finished

Training finished.
